In [1]:
import os

In [2]:
%pwd

'c:\\Users\\omarh\\OneDrive\\Desktop\\To do\\Projects\\SalesForecast\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\omarh\\OneDrive\\Desktop\\To do\\Projects\\SalesForecast'

In [17]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    model_path: Path
    test_data_path: Path
    metrics_file: Path

In [18]:
from salesRegressor.constants import *
from salesRegressor.utils.common import read_yaml, create_directories

In [19]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])
    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        
        create_directories([config.root_dir])
        
        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            model_path=config.model_path,
            test_data_path=config.test_data_path,
            metrics_file=config.metrics_file
            )

        return model_evaluation_config

In [20]:
import json
import numpy as np
import pandas as pd
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error

In [21]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    @staticmethod
    def rmspe_metric(y_true, y_pred):

        y_true = np.array(y_true)
        y_pred = np.array(y_pred)
        mask = y_true != 0
        return np.sqrt(np.mean(((y_true[mask] - y_pred[mask]) / y_true[mask]) ** 2))

    def evaluate(self):
        model = CatBoostRegressor()
        model.load_model(self.config.model_path)

        df_test = pd.read_csv(self.config.test_data_path, low_memory=False)

        X_test = df_test.drop(['Sales', 'Date'], axis=1)
        y_test = df_test['Sales']

        y_pred_log = model.predict(X_test)
        y_pred = np.expm1(y_pred_log)

        rmse = np.sqrt(mean_squared_error(np.expm1(y_test), y_pred))
        rmspe = self.rmspe_metric(np.expm1(y_test), y_pred)

        metrics = {
            "RMSE": float(rmse),
            "RMSPE": float(rmspe)
        }

        with open(self.config.metrics_file, "w") as f:
            json.dump(metrics, f, indent=4)

        print(f"Model Evaluation Completed. RMSE: {rmse:.4f}, RMSPE: {rmspe:.4f}")

In [22]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluator = ModelEvaluation(model_evaluation_config)
    model_evaluator.evaluate()
    
except Exception as e:
    raise e

[2025-12-02 11:22:23,232: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-12-02 11:22:23,234: INFO: common: yaml file: params.yaml loaded successfully]
[2025-12-02 11:22:23,235: INFO: common: created directory at: artifacts]
[2025-12-02 11:22:23,237: INFO: common: created directory at: artifacts/model_evaluation]
Model Evaluation Completed. RMSE: 460.1679, RMSPE: 0.0684
